In [ ]:
import pandas as pd, requests, datetime, json, pandas_datareader as web
from bs4 import BeautifulSoup

start_date = '30-04-2021' # Дата выгрузки

# Слева как получить площадку, справа, как получить значение
moex_indices_dict = {
    'RU000A1002D0': 'RU000A1002D0',
    'RU000A0JX1H4': 'RU000A0JX1H4',
    'RU000A101R66': 'ESGR',
    'RU000A100P44': 'SBRB',
    'RU000A1025V3': 'RUAL',
    'RU000A1030R1': 'PRIE'
}

# Слева индекс, справа, как должно быть в итоговом файле
moex_indices_alias = {
    'RU000A1002D0': 'PIF_DENEG',
    'RU000A0JX1H4': 'PIF_RUBUK',
    'RU000A101R66': 'ESGR',
    'RU000A100P44': 'PIF_SBRB',
    'RU000A1025V3': 'RUAL',
    'RU000A1030R1': 'PIF_PRIE'
}

In [15]:
def get_date(date): # преобразуем дату
    date = datetime.datetime.strptime(date, '%d-%m-%Y').date()
    return date

def get_primary_boardid(index): # Узнать площадку по индексу
    page = requests.get('https://iss.moex.com/iss/securities.xml?q=' + index)
    soup = BeautifulSoup(page.text, 'lxml')
    primary_boardid = soup.rows.row.get('primary_boardid')
    return primary_boardid

def get_value(index, date): # Получить значением индекса
    index_boardid = get_primary_boardid(index)
    df = web.DataReader(moex_indices_dict.get(index), 'moex', date, date)
    value = df[df.BOARDID == index_boardid]['ADMITTEDQUOTE'][0]
    return value

def get_string(date, code, currency, index_type, value): # Склейка строки
    value = str(value)
    date = str(date)
    getted_string = date +';'+ code +';'+ currency +';'+'%'+';'+ index_type +';'+ date +';advisory;'+ value +';'+ value
    return getted_string

In [16]:
data_moex = []
date = get_date(start_date) # получаем дату в нужном формате
moex_indices = list(moex_indices_dict.keys()) # список индексов для сбора

for index in moex_indices:
    index_type = 'b'
    currency = 'RUR'
    date_style = datetime.datetime.strftime(date, '%d/%m/%Y')
    code = moex_indices_alias.get(index)
    
    try: # Проверяем если значение за эту дату, иначе 0
        index_value = get_value(index, date)
    except:
        index_value = 0
    
    data_moex.append([date_style, date_style, code, currency, index_type, index_value, index_value,
                      get_string(date_style, code, currency, index_type, index_value)])

data_moex = pd.DataFrame(data_moex,
                         columns=['start', 'end','code', 'currency', 'index_type',
                                  'value_01', 'value_02', 'string'])

In [17]:
data_moex

,start,end,code,currency,index_type,value_01,value_02,string
0,30/04/2021,30/04/2021,PIF_DENEG,RUR,b,1191.000,1191.000,30/04/2021;PIF_DENEG;RUR;%;b;30/04/2021;adviso...
1,30/04/2021,30/04/2021,PIF_RUBUK,RUR,b,1792.000,1792.000,30/04/2021;PIF_RUBUK;RUR;%;b;30/04/2021;adviso...
2,30/04/2021,30/04/2021,ESGR,RUR,b,1384.800,1384.800,30/04/2021;ESGR;RUR;%;b;30/04/2021;advisory;13...
3,30/04/2021,30/04/2021,PIF_SBRB,RUR,b,1120.600,1120.600,30/04/2021;PIF_SBRB;RUR;%;b;30/04/2021;advisor...
4,30/04/2021,30/04/2021,RUAL,RUR,b,50.590,50.590,30/04/2021;RUAL;RUR;%;b;30/04/2021;advisory;50...
5,30/04/2021,30/04/2021,PIF_PRIE,RUR,b,10.049,10.049,30/04/2021;PIF_PRIE;RUR;%;b;30/04/2021;advisor...
